In [ ]:
import os
import gzip
import hashlib
import requests
import csv
from qnt.output_avro import convert_avro_record_to_output, convert_output_to_avro_record
from qnt.data import sort_and_crop_output

In [ ]:
# Feed API avro schema
feed_api = os.environ['STATAN_FEED_URL']

# Request authorization header (PUT YOUR PERSONAL TOKEN HERE)
access_token = os.environ['PERSONAL_ACCESS_TOKEN']

# Blockchain public API which should return all information about verification transactions and dates
blockchain_api = os.environ['STATAN_BLOCKCHAIN_URL']

In [ ]:
feed_api

In [ ]:
access_token

In [ ]:
blockchain_api

In [ ]:
# Fetching data from Blockchain API
blockchain_api_data = requests.get(blockchain_api).json()["result"]

# Creating empty arrays for storing blockchain data

# Blockchain transactions hashes
txs = []

# SHA256 root hashes generated from daily data
roots = []

# CSV texts of submissions (submission_id, daily_output_avro_hash)
csv_texts = []

# Dates of each CSV text fetched from the Feed API
dates = []

# Sorting Blockchain API data into arrays by its type
for data in blockchain_api_data:
    txs.append(data["tx"])
    roots.append(data["root"])
    csv_texts.append(data["data"])
    dates.append(data["date"])

In [ ]:
# Hashing the daily data with a SHA256 algorithm, and comparing it with the provided data from the Blockchain API
def compare_daily_data_hash_with_generated_hash(root, data):
    hash = hashlib.sha256(data.encode('utf-8')).hexdigest()
    return bool(True) if hash == root else bool(False)

# Verifying all dates and hashes
# as an argument need to pass root hashes and CSV texts
# the lengths of arrays should be equal
def compare_root_hashes_with_daily_data_hashes(roots, csv_texts):
    if (len(roots) == len(csv_texts)):
        i = 0
        while i < len(csv_texts):
            is_valid = compare_daily_data_hash_with_generated_hash(roots[i], csv_texts[i])
            result = "the same" if is_valid else "different"
            print("The hash of CSV data and root from " + dates[i] + " is " + result)
            i += 1
    else:
        print("The lengths should be equal!")

In [ ]:
# Step 1: Fetch the personal output data for certain date by submission_id
# Step 2: Hashing the output with SHA256
# Step 3: Comparing the output hash and the hash from Blockchain API

# Send a request to the feed API with the personal access token
def send_get_submision_output_request(submission_id):
    output = requests.get(feed_api + submission_id, headers={ "Authorization" : "token " + access_token })
    return output

# Generate the SHA256 hash by submission_id for certain date
def generate_output_hash(output, date):
    xarr = convert_avro_record_to_output(output.content)
    s = xarr.loc[date:date]
    s = sort_and_crop_output(s)
    avs = convert_output_to_avro_record(s)
    hash = hashlib.sha256(avs).hexdigest()
    return hash

# Generate the SHA256 hash from the output and verify with the hash from Blockchain API
def verify_outputs_of_date(date):
    data = csv_texts[dates.index(date)]
    csv_data = csv.DictReader(data.splitlines())
    for row in csv_data:
        submission_id = row["submission_id"]
        output = send_get_submision_output_request(submission_id)
        if (output.status_code == 200):
            avro_hash = row["daily_output_avro_hash"]
            generated_hash = generate_output_hash(output, date)
            result = "valid" if generated_hash == avro_hash else "invalid"
            print("ID " + submission_id + " at " + date + " is " + result)

# Verify all outputs from all dates
def verify_outputs_all_dates(dates):
    print("Starting the process ...")
    for date in dates:
        verify_outputs_of_date(date)
    print("Done!")

In [ ]:
# Verification of the generated SHA256 hash of the daily data and provided SHA256 hashes
verify_outputs_all_dates(dates)

In [ ]:
# Verification of blockchain based roots and the daily data
compare_root_hashes_with_daily_data_hashes(roots, csv_texts)